In [1]:
#coding=utf-8
import matplotlib.pyplot as plt
import numpy as np  
from sklearn.model_selection import train_test_split
import keras
from keras import models
from keras.models import Sequential  
from keras.layers import *  
from keras.utils.np_utils import to_categorical  
from keras.models import Model
import cv2
import os
import gdal
import tensorflow as tf
from sklearn import tree, svm, naive_bayes,neighbors
from sklearn.ensemble import BaggingClassifier, AdaBoostClassifier, RandomForestClassifier,  GradientBoostingClassifier

D:\Python37\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
D:\Python37\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
D:\Python37\lib\site-packages\numpy\.libs\libopenblas.TXA6YQSD3GCQQC22GEQ54J2UDCXDXHWN.gfortran-win_amd64.dll
  stacklevel=1)
Using TensorFlow backend.
D:\Python37\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\Python37\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\Python37\lib\site-packages\tensorflow\python\framework\dt

In [2]:
filepath = r"E:\project\images\researchImage\resize"
os.chdir(filepath)
# read the images
def readimg(name):
    img = gdal.Open(os.path.join(filepath, name))
    img_b = img.GetRasterBand(1).ReadAsArray()
    img_g = img.GetRasterBand(2).ReadAsArray()
    img_r = img.GetRasterBand(3).ReadAsArray()
    img_n = img.GetRasterBand(4).ReadAsArray()
    img = np.dstack((img_b,img_g,img_r,img_n))
    print(img.shape)
    width=img.shape[0]
    height=img.shape[1]
    img.resize((img.shape[0]*img.shape[1],4))
    return img,width,height

In [12]:
img1,w1,h1=readimg("000326.tif")
img1=img1[100000:101000,:]
img2,w2,h2=readimg("040121.tif")
img2=img2[100000:101000,:]
img3,w3,h3=readimg("090618.tif")
img3=img3[100000:101000,:]
img4,w4,h4=readimg("181218.tif")
img4=img4[100000:101000,:]

(1342, 1358, 4)
(1342, 1358, 4)
(1342, 1358, 4)
(1241, 1474, 4)


In [13]:
def gendata(data,rate,num_0,num_1,num_2,num_3):
    X = data[:,:]
    label_0=np.zeros((num_0))
    label_1=np.ones((num_1))
    label_2=np.ones((num_2))+1
    label_3=np.ones((num_3))+2
    labels=np.concatenate((label_0,label_1,label_2,label_3),axis=0)
    labels=labels.reshape(labels.shape[0],1)
    train_dataset=np.concatenate((X,labels),axis=1)
    np.random.shuffle(train_dataset)
    train_dataset=train_dataset[:,:]
    #split train_data and test_data
    X=train_dataset[:,:4]
    labels=train_dataset[:,-1:]
    X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=rate)
    print("X_train shape:",X_train.shape)
    print("X_test shape:",X_test.shape)
    print("y_train shape:",y_train.shape)
    print("y_test shape:",y_test.shape)
    return X_train,X_test,y_train,y_test
clfs = {
         'decision_tree':tree.DecisionTreeClassifier(),
         #'naive_gaussian': naive_bayes.GaussianNB(),
         #'naive_mul':naive_bayes.MultinomialNB(),
         'K_neighbor' : neighbors.KNeighborsClassifier(),
         'bagging_knn' : BaggingClassifier(neighbors.KNeighborsClassifier(), max_samples=0.5,max_features=0.5),
         'bagging_tree': BaggingClassifier(tree.DecisionTreeClassifier(), max_samples=0.5,max_features=0.5),
         'random_forest' : RandomForestClassifier(n_estimators=50),
         'gradient_boost' : GradientBoostingClassifier(n_estimators=50, learning_rate=1.0,max_depth=1, random_state=0)
       }

def try_different_method(clf,X_train,X_test,y_train,y_test):
    clf.fit(X_train,y_train.ravel())
    score = clf.score(X_test,y_test.ravel())
    print('the score is :', score)

In [14]:
data=np.concatenate((img1,img2,img3,img4),axis=0)
X_train,X_test,y_train,y_test=gendata(data,0.2,1000,1000,1000,1000)

X_train shape: (3200, 4)
X_test shape: (800, 4)
y_train shape: (3200, 1)
y_test shape: (800, 1)


In [15]:
for clf_key in clfs.keys():
    print('the classifier is :',clf_key)
    clf = clfs[clf_key]
    try_different_method(clf,X_train,X_test,y_train,y_test)

the classifier is : decision_tree
the score is : 0.93875
the classifier is : K_neighbor
the score is : 0.94625
the classifier is : bagging_knn
the score is : 0.86
the classifier is : bagging_tree
the score is : 0.8525
the classifier is : random_forest
the score is : 0.95875
the classifier is : gradient_boost
the score is : 0.9325
